In [9]:
# Importaciones estándar de Python
import json
from datetime import datetime

# Importaciones de librerías de terceros
import requests
import pandas as pd
from unidecode import unidecode

# Función auxiliar coa
def coa(df, a):
    e = ""
    try:
        e = df[a][0]
    except:
        e = ""
    return e

# Función auxiliar dts
def dts(matchID):
    bandera = True
    cont = 0
    while bandera and cont < 3:
        try:
            response = requests.get(
                "https://www.sofascore.com/api/v1/unique-tournament/17/season/61627/team-events/total",
                proxies={
                    "http": "http://ef9cc2c89bd6437da0a54566ef31b81c:@proxy.crawlera.com:8011/",
                    "https": "http://ef9cc2c89bd6437da0a54566ef31b81c:@proxy.crawlera.com:8011/",
                },
                verify='C:\\Users\\frato\\anaconda3\\lib\\site-packages\\certifi\\cacert.pem',
                timeout=10
            )
            data_json = json.loads(response.text)
            df = pd.json_normalize(data_json)
            return df
        except:
            cont += 1
            continue
    return pd.DataFrame()

# Función principal datos_partido
def datos_partido(matchID):
    col_parts = ['pais', 'torneo', 'temporada', 'nombre temp', 'jornada', 'dia', 'hora', 'estadio pais',
                 'estadio ciudad', 'estadio', 'arbitro_ID', 'arbitro', 'arbitro pais',
                 "Local_ID", "Local pais", "Local_nom", "Local",
                 'Local_DTID', 'Local_DT', 'Local_DT pais',
                 'visita_ID', 'visita_pais', 'visita_nom', 'visita',
                 'visita_DTID', 'visita_DT', 'visita_DT pais',
                 'GF', 'GC']
    bandera = True
    while bandera:
        try:
            response = requests.get(
                "https://www.sofascore.com/api/v1/unique-tournament/17/season/61627/team-events/total",
                proxies={
                    "http": "http://ef9cc2c89bd6437da0a54566ef31b81c:@proxy.crawlera.com:8011/",
                    "https": "http://ef9cc2c89bd6437da0a54566ef31b81c:@proxy.crawlera.com:8011/",
                },
                verify='C:\\Users\\frato\\anaconda3\\lib\\site-packages\\certifi\\cacert.pem',
                timeout=10
            )
            data_json = json.loads(response.text)
            df = pd.json_normalize(data_json)
            bandera = False
        except:
            continue

    if coa(df, 'event.homeScore.current') == "":
        return None

    # managers = dts(matchID)
    torneo_pais = unidecode(coa(df, 'event.tournament.category.name'))
    torneo = unidecode(coa(df, 'event.tournament.name'))
    temp = coa(df, 'event.season.year')
    temp_name = coa(df, 'event.season.name')
    jornada = coa(df, 'event.roundInfo.round')
    fecha = datetime.fromtimestamp(df['event.startTimestamp'][0])
    dia = fecha.date()
    hora = fecha.time()

    estadio_pais = unidecode(coa(df, 'event.venue.country.name'))
    estadio_ciudad = unidecode(coa(df, 'event.venue.city.name'))
    estadio_name = unidecode(coa(df, 'event.venue.stadium.name'))

    arb_name = unidecode(coa(df, 'event.referee.name'))
    arb_id = coa(df, 'event.referee.id')
    arb_pais = unidecode(coa(df, 'event.referee.country.name'))

    local_id = df['event.homeTeam.id'][0]
    local_pais = unidecode(df['event.homeTeam.country.name'][0])
    local_nom = unidecode(df['event.homeTeam.nameCode'][0])
    local = unidecode(df['event.homeTeam.name'][0])
    GL = coa(df, 'event.homeScore.current')

    # local_DTID = coa(managers, 'homeManager.id')
    # local_DT = unidecode(coa(managers, 'homeManager.name'))
    local_DT_nac = ""

    visita_id = df['event.awayTeam.id'][0]
    visita_pais = unidecode(df['event.awayTeam.country.name'][0])
    visita_nom = unidecode(df['event.awayTeam.nameCode'][0])
    visita = unidecode(df['event.awayTeam.name'][0])
    GV = coa(df, 'event.awayScore.current')

    # visita_DTID = coa(managers, 'awayManager.id')
    # visita_DT = unidecode(coa(managers, 'awayManager.name'))
    visita_DT_nac = ""

    print(torneo_pais + " - " + temp_name + ", Fecha " + str(jornada))
    print(fecha)
    print(estadio_name + " (" + estadio_ciudad + ", " + estadio_pais + ")")
    print("[" + str(arb_id) + "] " + arb_name + " (" + arb_pais + ")")
    print("[" + str(local_id) + local_nom + "] " + local)
    # print("[" + str(local_DTID) + "] " + local_DT)
    print("[" + str(visita_id) + visita_nom + "] " + visita)
    # print("[" + str(visita_DTID) + "] " + visita_DT)
    print(local + " " + str(GL) + " - " + str(GV) + " " + visita + "\n")

    columnas = [torneo_pais, torneo, temp, temp_name, jornada, dia, hora, estadio_pais,
                estadio_ciudad, estadio_name, arb_id, arb_name, arb_pais,
                local_id, local_pais, local_nom, local,
                # local_DTID, local_DT, local_DT_nac,
                visita_id, visita_pais, visita_nom, visita,
                # visita_DTID, visita_DT, visita_DT_nac,
                GL, GV]

    dict3 = dict(zip(col_parts, columnas))
    match_data = pd.DataFrame([dict3], columns=col_parts)
    # match_data.insert(0, 'matchID', matchID)
    match_data.reset_index(drop=True, inplace=True)
    return match_data


Error al conectar con la API: "None of [Index(['startTimestamp', 'homeTeam.name', 'awayTeam.name', 'homeScore.current',\n       'awayScore.current', 'roundInfo.round', 'venue.name', 'status.type'],\n      dtype='object')] are in the [columns]"


In [8]:
import LanusStats as ls  
sofascore = ls.SofaScore()

In [29]:

# Importaciones estándar de Python
import json
from datetime import datetime

# Importaciones de librerías de terceros
import requests
import pandas as pd
from unidecode import unidecode

# Función para recolectar todos los partidos
def obtener_fixture_premier():
    # URL del endpoint del fixture
    url = "https://www.sofascore.com/api/v1/unique-tournament/17/season/61627/team-events/total"

    # Headers opcionales (simular un navegador)
    headers = {
        "user-agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.6 Mobile/15E148 Safari/604.1",
        "accept": "*/*"
    }

    try:
        # Realizar la solicitud GET
        response = requests.get(url, headers=headers, timeout=10)

        # Validar que la solicitud fue exitosa
        if response.status_code == 200:
            # Parsear la respuesta JSON
            data_json = response.json()

            # Normalizar los datos en un DataFrame
            partidos = pd.json_normalize(data_json['events'])

            # Seleccionar columnas relevantes
            partidos = partidos[[
                'startTimestamp',                 # Fecha y hora en formato Unix
                'homeTeam.name',                  # Equipo local
                'awayTeam.name',                  # Equipo visitante
                'homeScore.current',              # Goles del equipo local
                'awayScore.current',              # Goles del equipo visitante
                'roundInfo.round',                # Jornada
                'venue.name',                     # Estadio
                'status.type',                    # Estado del partido (ej. FINISHED, SCHEDULED)
            ]]

            # Convertir la fecha y hora al formato legible
            partidos['startTimestamp'] = pd.to_datetime(partidos['startTimestamp'], unit='s')

            # Renombrar columnas para facilitar la lectura
            partidos.rename(columns={
                'startTimestamp': 'Fecha y Hora',
                'homeTeam.name': 'Equipo Local',
                'awayTeam.name': 'Equipo Visitante',
                'homeScore.current': 'Goles Local',
                'awayScore.current': 'Goles Visitante',
                'roundInfo.round': 'Jornada',
                'venue.name': 'Estadio',
                'status.type': 'Estado'
            }, inplace=True)

            # Retornar el DataFrame para procesamiento adicional
            return partidos
        else:
            print(f"Error en la solicitud: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error al conectar con la API: {e}")
        return None

# Llamar a la función y guardar el fixture
fixture_premier = obtener_fixture_premier()

# Guardar el fixture como CSV
if fixture_premier is not None:
    fixture_premier.to_csv("fixture_premier_league.csv", index=False)
    print("Fixture guardado en 'fixture_premier_league.csv'")

Error al conectar con la API: 'events'


In [21]:
import requests
import pandas as pd

def obtener_eventos_premier_league(url):
    """
    Obtiene todos los eventos de la Premier League desde la API.
    """
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data.get('tournamentTeamEvents', {})
    else:
        print(f"Error al obtener datos: {response.status_code}")
        return {}

def procesar_eventos(eventos):
    """
    Procesa los eventos y devuelve un DataFrame con la información relevante.
    """
    data = []
    for key, matches in eventos.items():
        # Asegúrate de que matches sea una lista
        if isinstance(matches, list):
            for match in matches:
                evento = {
                    'tournament': match['tournament']['name'],
                    'home_team': match['homeTeam']['name'],
                    'away_team': match['awayTeam']['name'],
                    'home_score': match['homeScore']['current'],
                    'away_score': match['awayScore']['current'],
                    'status': match['status']['description'],
                    'date': match['startTimestamp']
                }
                data.append(evento)
        else:
            print(f"Advertencia: se esperaba una lista, pero se encontró: {type(matches)}")
    
    return pd.DataFrame(data)

def main():
    url = "https://www.sofascore.com/api/v1/unique-tournament/17/season/61627/team-events/total"
    eventos = obtener_eventos_premier_league(url)
    if eventos:
        df_eventos = procesar_eventos(eventos)
        print(df_eventos.head())  # Muestra las primeras filas del DataFrame

if __name__ == '__main__':
    main()

Advertencia: se esperaba una lista, pero se encontró: <class 'dict'>
Empty DataFrame
Columns: []
Index: []


In [23]:
import LanusStats as ls  
fbref = ls.Fbref()

In [24]:
posiciones = fbref.get_tournament_table("https://fbref.com/en/comps/9/Premier-League-Stats")
print(posiciones)

    Rk            Squad  MP   W  D   L  GF  GA  GD  Pts  Pts/MP    xG   xGA  \
0    1        Liverpool  13  11  1   1  26   8  18   34    2.62  26.4  11.3   
1    2          Arsenal  13   7  4   2  26  14  12   25    1.92  21.7  13.8   
2    3          Chelsea  13   7  4   2  26  14  12   25    1.92  24.1  17.1   
3    4         Brighton  13   6  5   2  22  17   5   23    1.77  19.0  21.0   
4    5  Manchester City  13   7  2   4  22  19   3   23    1.77  24.9  18.8   
5    6  Nott'ham Forest  13   6  4   3  16  13   3   22    1.69  16.1  12.9   
6    7        Tottenham  13   6  2   5  28  14  14   20    1.54  24.6  16.7   
7    8        Brentford  13   6  2   5  26  23   3   20    1.54  21.7  21.5   
8    9   Manchester Utd  13   5  4   4  17  13   4   19    1.46  19.4  19.0   
9   10           Fulham  13   5  4   4  18  18   0   19    1.46  19.5  14.8   
10  11    Newcastle Utd  13   5  4   4  14  14   0   19    1.46  18.0  17.2   
11  12      Aston Villa  13   5  4   4  19  22  -3  

In [29]:
import warnings
warnings.filterwarnings("ignore")
import requests
from bs4 import BeautifulSoup, Comment
import pandas as pd
from datetime import datetime
import time
import numpy as np


In [36]:
def get_all_teams_season_stats(self, league="Premier League", season="2023-2024", save_csv=False, stats_vs=False, change_columns_names=False, add_page_name=False):
        
        today = datetime.now().strftime('%Y-%m-%d')
        data = pd.DataFrame()

        for stat in self.possible_stats:
            placeholder = self.get_teams_season_stats(f'{stat}',league, season, False, stats_vs, change_columns_names, add_page_name)
            data = pd.concat([data, placeholder], axis=1)
        
        if save_csv and stats_vs:
            data.to_csv(f'{league} - {stat} - team vs stats - {today}.csv')
        elif save_csv: 
            data.to_csv(f'{league} - {stat} - team stats - {today}.csv')

        return data

In [4]:
import requests
import pandas as pd
import os
import time

def get_matches(tournament_id="17", season_id="61627", path="https://www.sofascore.com/api/v1/unique-tournament/17/season/61627/team-events/total"):
   base_url = f'https://www.sofascore.com/api/v1/unique-tournament/{tournament_id}/season/{season_id}'

   # check if tournament has any past/future matches
   if requests.head(f'{base_url}/events/{path}/0').status_code != 200:
      return []

   result = []
   i = 0
   hasNextPage = True
   while hasNextPage:
      url = f'{base_url}/events/{path}/{i}'
      response = requests.get(url)
      data = response.json()
      hasNextPage = data.get('hasNextPage')
      result.extend(data.get('events', []))
      i += 1

   return result


def get_all_events(tournament_id = "", season_id):
   past_matches = get_matches(tournament_id, season_id, 'last')
   future_matches = get_matches(tournament_id, season_id, 'next')

   # sort past matches by date (future matches already sorted)
   past_matches.sort(key=lambda x: x['startTimestamp'])

   return  past_matches + future_matches
   


def get_match_incidents(event_id):
   url = f'https://www.sofascore.com/api/v1/event/{event_id}/incidents'
   response = requests.get(url)
   data = response.json()
   return data.get('incidents', [])


def main():
   tournament_id = 649
   season_id = 58388
   matches = get_all_events(tournament_id, season_id)

   data = []
   for match in matches:
      incidents = get_match_incidents(match['id'])
      data.append([
         match['tournament']['name'],
         match['homeTeam']['name'],
         match['homeScore'].get('current'),
         match['awayTeam']['name'], 
         match['awayScore'].get('current'),
         time.strftime('%x %X', time.localtime(int(match['startTimestamp']))),
         match['status']['description'],
         ', '.join( f"{i['time']}'" for i in incidents if i['incidentType'] == 'goal')
      ])

   headers = ['Tournament', 'Home Team', 'Home Score', 'Away Team', 'Away Score', 'Start Date', 'Status', 'Goal Times']

   df = pd.DataFrame(data, columns=headers)
   df.to_excel("output.xlsx", index=False)


if __name__ == '__main__':
   main()

SyntaxError: non-default argument follows default argument (2007005963.py, line 27)

In [2]:
import pandas as pd

# Load the Excel file
file_path = 'output.xlsx'  # Update the path if necessary
try:
    # If the Excel file contains multiple sheets
    excel_data = pd.ExcelFile(file_path)

    # Display available sheet names
    print("Sheets in the Excel file:", excel_data.sheet_names)

    # Load the first sheet as a DataFrame
    df = excel_data.parse(sheet_name=0)  # Change sheet_name if needed (e.g., "Sheet1")
    print("First sheet loaded successfully!")

    # Display the first few rows of the sheet
    print(df.head())

except FileNotFoundError:
    print("File not found. Ensure the path to 'output.xlsx' is correct.")
except Exception as e:
    print(f"An error occurred: {e}")


Sheets in the Excel file: ['Sheet1']
First sheet loaded successfully!
Empty DataFrame
Columns: [Tournament, Home Team, Home Score, Away Team, Away Score, Start Date, Status, Goal Times]
Index: []


In [5]:
import requests
import pandas as pd
import time


def get_matches(tournament_id="17", season_id="61627", match_type="last"):
    """
    Fetch matches for a tournament season based on type ('last' for past, 'next' for future).
    """
    base_url = f'https://www.sofascore.com/api/v1/unique-tournament/{tournament_id}/season/{season_id}/events/{match_type}'
    result = []
    page = 0

    while True:
        url = f'{base_url}/{page}'
        response = requests.get(url)
        if response.status_code != 200:
            break
        data = response.json()
        events = data.get('events', [])
        if not events:
            break
        result.extend(events)
        if not data.get('hasNextPage'):
            break
        page += 1

    return result


def get_all_events(tournament_id, season_id):
    """
    Fetch all past and future matches for a tournament season.
    """
    # Get past matches
    print("Fetching past matches...")
    past_matches = get_matches(tournament_id, season_id, match_type="last")

    # Get future matches
    print("Fetching future matches...")
    future_matches = get_matches(tournament_id, season_id, match_type="next")

    # Sort past matches by date
    past_matches.sort(key=lambda x: x['startTimestamp'])

    return past_matches + future_matches


def get_match_incidents(event_id):
    """
    Fetch match incidents like goals, cards, etc., for a specific match.
    """
    url = f'https://www.sofascore.com/api/v1/event/{event_id}/incidents'
    response = requests.get(url)
    if response.status_code != 200:
        return []
    data = response.json()
    return data.get('incidents', [])


def main():
    # Specify tournament and season IDs
    tournament_id = "17"  # Premier League ID
    season_id = "61627"  # Example season ID

    # Fetch all matches (past and future)
    matches = get_all_events(tournament_id, season_id)
    print(f"Total matches fetched: {len(matches)}")

    # Process match data into a structured format
    data = []
    for match in matches:
        event_id = match['id']
        incidents = get_match_incidents(event_id)

        # Extract goal times from incidents
        goal_times = [f"{incident['time']}'" for incident in incidents if incident.get('incidentType') == 'goal']

        # Append match details to the data list
        data.append([
            match['tournament']['name'],  # Tournament name
            match['homeTeam']['name'],   # Home team
            match['homeScore'].get('current', ""),  # Home score
            match['awayTeam']['name'],   # Away team
            match['awayScore'].get('current', ""),  # Away score
            time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(int(match['startTimestamp']))),  # Match start time
            match['status']['description'],  # Match status (e.g., Ended, Not Started)
            ', '.join(goal_times)  # Goal times
        ])

    # Define headers for the DataFrame
    headers = ['Tournament', 'Home Team', 'Home Score', 'Away Team', 'Away Score', 'Start Date', 'Status', 'Goal Times']

    # Create a DataFrame
    df = pd.DataFrame(data, columns=headers)

    # Save to Excel
    output_file = "output.xlsx"
    df.to_excel(output_file, index=False)
    print(f"Data saved to {output_file}")


if __name__ == '__main__':
    main()


Fetching past matches...
Fetching future matches...
Total matches fetched: 0
Data saved to output.xlsx
